In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [148]:
import time
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

In [2]:
es.indices.get_alias("*")

{u'.apm-agent-configuration': {u'aliases': {}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'montevideo': {u'aliases': {}}}

In [57]:
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)

In [109]:
text = 'Siento mucha tristeza al ver esto en Montevideo. Una persona compartió esta imagen que ocurrió hace minutos en la esquina de Salto y (Bernabé) Rivera. La cantidad de personas en situación de calle creció en la capital. Hay planes para mejorar esta realidad, pero duele verla.'

In [94]:
text = 'Mucha basura suelta por Varela frente al Policial'

In [161]:
text = 'En Montevideo podés ir a Porongos esquina Blandengues Upside-down face'

In [182]:
text = '18 de Julio esquina Aquiles Lanza, Montevideo, Uruguay'

In [187]:
text = 'Roban estación de servicio  esquina Montevideo shopping. Hace una semana robaron una joyería a 50 metros de esa estación de servicio. Lugar de mucho tránsito de personas y vehículos. Lleno de camaras de video vigilancia y PADO Man police officerDown pointing backhand index'

In [195]:
text = 'En esta foto estas en que lugar de Europa, amorosa? Ésta estuvo en el cerro de montevideo, calle Venezuela esquina Bogotá....Jaaaa....sos una genia! Smiling face with open mouth and smiling eyes'

In [202]:
text = 'Construction sign👷🏻‍| #MontevideoMejora Round pushpin Av. Italia de Gallinal a Bolivia, luce más amplia y linda Bus Un carril más para el bus Splashing sweat symbol Mejor captación de aguas pluviales 🅿 Refugios Bicycle Bicisenda Electric light bulb 40 nuevas columnas de alumbrado y 80 luminarias LED Más info: Mobile phone with rightwards arrow at left http://bit.ly/3bbb3jA'

In [216]:
text = '@quejasyaEn la esquina de Montevideo Shopping hay una "montaña" de basura (Galarza y Tiburcio Gómez)'

In [219]:
text = 'Cheering megaphone Modificación de paradas para líneas 147 y 148 hacia afuera Calendar Desde el 5 de marzo Cross mark Se suprime la parada de Paysandú esquina Tristán Narvaja Hourglass Funciona provisoriamente en Tristán Narvaja esquina Cerro Largo'

In [230]:
text = 'Un hecho histórico en la Medicina Mundial. Ciudad de Montevideo. Esquina de Colonia y Arenal Grande. CASMU 1'

In [233]:
text = 'Cno. de las Tropas esquina Verdún'

In [236]:
text = '¡Hola! Para este tipo de errores en el registro debes acudir a nuestra oficina, allí te darán la información y la ayuda necesaria para corregir este error y completar tu proceso. Recuerda que nuestra oficina está ubicada en Montevideo, José Ellauri 938, esquina Benito Lamas. 🚴🏽Sparkling heart'

In [239]:
text = '🚧❌ | MEDIA CALZADA 📍 26 DE MARZO entre L.A. de Herrera y Marco Bruto 🕖 De 7 a 16 h  📍 JOSÉ ELLAURI esquina Leyenda Patria 🕖 De 7 a 17 h  📍 LAGUNILLAS esquina Joaquín Núñez 🕖 De 8 a 18 h  📍 8 DE OCTUBRE desde 18 de Julio hasta Colonia 🕖 De 8 a 18 h'

In [245]:
text = 'Speaker with three sound waves “Durazno y Convención” es una canción de 1984, que hoy es un tema que se puede escuchar mismo en la esquina sobre la que habla la letraRound pushpin  Ya que es un punto de Montevideo Sonoro Mobile phone Headphone'

In [248]:
text = 'En la ciudad de Montevideo, hace ya bastante tiempo, están dibujados los hongos de Super Mario Bros. En avenida Uruguay esquina Arenal Grande, está dibujado un hongo rojo. En avenida San Martín esquina Libres, está dibujado un hongo verde. Smiling face with heart-shaped eyes'

In [251]:
text = 'Alquiler anual. Centro de Montevideo 1 dormitorio $19.000 Héctor Gutiérrez Ruiz esquina San José Coordine visitas al 094283539 Llamadas o wsp'

In [254]:
text = '¿Sos marica que usás vestido? Utilizando ésta frase tres efectivos de la Guardia Republicana apalearon a Joel, artista callejero que trabaja habitualmente en Millán y Garzón.'

In [255]:
results = ic.analyze(index="montevideo", body=
{
  "text": text, 
    "analyzer":"calle_analyzer"
})
for result in results['tokens']:
    print(result['token'])

sos
marica
usas
vestido
utilizando
frase
tres
efectivos
guardia
republicana
apalearon
joel
artista
callejero
trabaja
habitualmente
millan
garzon


In [242]:
#@timing
def search(text):
    return es.search(index="montevideo", body=
        {
            "from" : 0, "size" : 500,
              "query": {
                "simple_query_string" : {
                    "query": text,
                    "analyzer": "calle_analyzer"
                }
              }
        })['hits']['hits']

In [212]:
#@timing
def search_geo(id):
    return es.search(index="montevideo", body=
    {
        "from" : 0, "size" : 100,
        "query" : {
            "bool": {
                "filter": {
                    "geo_shape": {
                        "geometry": {
                            "indexed_shape": {
                                "index": "montevideo",
                                "id": id,
                                "path": "geometry"
                            }
                        }
                    }
                }
            }
        }
    })['hits']['hits']

In [ ]:
def tryToMatchLines():
    

In [257]:
@timing
def complete_search(text):
    match_dict = {}
    matchs = []
    state = {}
    results = search(text)
    filtered_results = []
    
    for result in results:
        if (result['_source']['type'] != 'limites_barrios'):
            state[result['_id']] = {"_source":result['_source'],"_score":result['_score']}
            filtered_results.append(result)

    for result in filtered_results:
        nombre = result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
        if nombre in match_dict.keys() or nombre in match_dict.values():
            continuematch_dict
        #print('{}\t\t{}\t\t{} {}'.format(result['_id'],nombre,result['_source']['geometry']['type'],result['_score']))
        for geo_result in search_geo(result["_id"]):
            geo_nombre = 'NO_NAME'
            if geo_result['_source'].get('nombre',None):
                geo_nombre = geo_result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
            elif geo_result['_source'].get('aliases',None):
                geo_nombre = geo_result['_source']['aliases'].encode('ascii', 'ignore').decode('ascii')
            if geo_result['_id'] != result['_id'] and geo_nombre != nombre:
                #print('\t{}\t{}\t\t{}'.format(geo_result['_id'], geo_nombre, geo_result['_source']['geometry']['type']))
                if geo_result['_id'] in state.keys():
                    print ('MATCH: {}\t{}\t/\t{}\t{}'.format(nombre,result['_score'],geo_nombre,state[geo_result['_id']]['_score']))
                    match_dict[nombre] = geo_nombre
                    match_dict[geo_nombre] = nombre
                    matchs.append([result["_id"],geo_result['_id']])
    return (matchs,state)

In [258]:
(matchs,state) = complete_search(text)

MATCH: AV MILLAN	11.369972	/	AV GRAL EUGENIO GARZON	8.961531
complete_search function took 328.241 ms


In [215]:
matchs

[]

In [98]:
results = es.search(index="montevideo", body=
{
    "from" : 0, "size" : 30,
    "query" : {
        "query_string" : {
            "query" : text,
            "analyzer": "calle_analyzer"
        }
    }
})
for result in results['hits']['hits']:
    print(result['_score'],result['_id'],result['_source']['type'],result['_source']['nombre'], result['_source']['geometry']['type'])

(6.5457935, u'v_mdg_vias-287716', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-7084344', u'v_mdg_vias', u'WENCESLAO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-6433252', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-6433297', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-287853', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.527852, u'geonames_uy_montevideo-7211405', u'geonames_uy_montevideo', u'Hospital Policial Inspector General Uruguay Genta', u'Point')
(5.572977, u'v_mdg_vias-7362536', u'v_mdg_vias', u'JUAN CRUZ VARELA', u'LineString')
(5.572977, u'v_mdg_vias-7595328', u'v_mdg_vias', u'AV JACOBO VARELA', u'LineString')
(5.572977, u'v_mdg_vias-7595334', u'v_mdg_vias', u'AV JACOBO VARELA', u'LineString')
(5.572977, u'v_mdg_vias-6373779', u'v_mdg_vias', u'AV JACOBO VARELA', u'LineString')
(5.572977, u'v_mdg_vias-295791', u'v_mdg_vias', u'JUAN CRUZ VARELA', u'LineString')
(5.572977, 

In [99]:

for result in results_2:


(0.0, u'v_mdg_vias-287716', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(0.0, u'v_mdg_vias-287824', u'v_mdg_vias', u'ANZANI', u'LineString')
(0.0, u'limites_barrios-10', u'limites_barrios', u'PQUE BATLLE VILLA DOLORES', u'Polygon')
